In [1]:
from AnalyticNoise2 import SwfNoise
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as pl

In [ ]:
def StDevHists(obj):
    
    numBands = len(obj.bands)
    for i in range(numBands):
        band_i = obj.bands[i]
        varMat_i = obj.rrsVarDict[band]
        
        for j in range(numBands+1):
            std_ij = np.sqrt(varMat_i[:,j])
            plt.subplot(331+j)
            rng=(np.nanmin(std_ij),np.nanmax(std_ij)
            plt.hist(std_ij,bins=100,color=)

In [ ]:
def CleanPlot(rrsPartVar,rrs,isSTD=False):
    
    if isSTD:
        rrsPartStd = rrsPartVar
    else:
        rrsPartStd= np.sqrt(rrsPartVar)
    rrsN0Idx = np.nonzero(rrs)[0]
    rrsPartStdPerc=100*rrsPartStd[rrsN0Idx]/rrs[rrsN0Idx]
    rrsPartStdPercClean = np.ma.masked_invalid(rrsPartStdPerc[rrsPartStdPerc>0])
    
    return rrsPartStdPercClean

def UncPartialContribPlots(rrsVar,rrsBsl,fTplWdHt=(18,15),**kwarg):
    """ I expect a 9 col. rrsVar w/ the first 8 are the 
        contribution of each band perturbation;
        the 9th is the aggregated effect on uncertainty.
        Abscissae units are in percentage points of std relative to 
        baseline rrs."""
    figname = 'Fig'
    
    if "figNum" in kwarg:
        figname += str(kwarg['figNum'])
    if  "figName" in kwarg:
        figname = kwarg['figName']
    if "saveDir" in kwarg:
        figname = kwarg['saveDir'] + figname
        
    figname+='.pdf'
    
    bins = 100
    if "bins" in kwarg:
        bins = kwarg['bins']
    logBin = False
    if "logbin" in kwarg:
        logBin = kwarg['logbin']
    
    if "useSTD" in kwarg:
        useSTD = kwarg['useSTD']
    else:
        useSTD = False
    
    if "useColors" in kwarg:
        colDict = kwarg['useColors']
        useCol = True
    else:
        useCol = False
    
    bands = [412,443,490,510,555,670,765,865]
    
    plt.figure(figsize=fTplWdHt)
    
    for i in range(9):
        plt.subplot(331+i)
        
        stdPerc = CleanPlot(rrsVar[:,i],rrsBsl,isSTD=useSTD)
        if "rangeDict" in kwarg:
            rangeDict = kwarg["rangeDict"]
            rangE = rangeDict[i]
        else:
            rangE = [0,1]
        stdPerc=stdPerc[stdPerc<rangE[1]]
        #plt.hist(stdPerc,range=rangE,bins=100,normed=True)
        if useCol and i <8:
            col = colDict[bands[i]]
            sns.distplot(stdPerc,bins=bins,kde=False,
                         color=col,hist_kws={'histype':'stepfilled'})
        sns.distplot(stdPerc,bins=bins,kde = False,hist_kws={'histtype':'stepfilled'})
        
        if i>5:
            plt.xlabel(r'$\sigma_{Rrs}(\%)$',fontsize=16)
        plt.tick_params(labelsize=14)
        if logBin:
            plt.gca().set_xscale("log")
        if i<8:
            plt.title(r'$\deltaLt_{%d}$'% bands[i])
        else:
            plt.title('Aggregate Uncertainty')
            
    plt.savefig(figname)

def UncAggPartContribPlots(rrsVar,rrsBsl,
                           fTplWdHt=(13,8),useLog = False,
                           **kwarg):

    if "range" in kwarg:
        plotRange = kwarg["range"]
    else:
        plotRange = [0,10]
        
    plt.figure(figsize=fTplWdHt)
    #ax = plt.subplot(1,2,1) 
    #ax2 = ax.twinx()
    
    if "binNums" in kwarg:
        binNums = kwarg["binNums"]
    
    else:
        binNums = 100
        
    if "colDict" in kwarg:
        colDict = kwarg["colDict"]
        keys = colDict.keys()
        useColors=True;
    else:
        useColors=False
    
    barwid = float(plotRange[1])/binNums
    accumVecHist = np.zeros((binNums,))
    
    for i in range(8):
        
        stdPerc = CleanPlot(rrsVar[:,i],rrsBsl)
        if useLog:
            
            binSet = np.logspace(plotRange[0],plotRange[1],num = binNums+1)
            print binSet
            stdPercHist,binEdges = np.histogram(stdPerc,range=plotRange,bins=binSet,density=True)
          
        else:
            
            stdPercHist,binEdges=np.histogram(stdPerc,range=plotRange,bins=binNums,density=True)
        
        
        print binEdges
        print np.diff(binEdges)
        print stdPercHist   
        binCenters = 0.5*(binEdges[1:]+binEdges[:-1])
        print np.diff(binCenters)
        # ax.bar(binEdges[:-1],stdPercHist,width = barwid)
        accumVecHist += stdPercHist
        
        if useColors:
            key = keys[i]
            plt.hist(binEdges,accumVecHist,histtype='step',label=str(i))
            # plt.plot(binCenters,accumVecHist,label=key,color=colDict[key])
        else:
            plt.plot(binCenters,accumVecHist,histtype='step',label=str(i))
        
        plt.legend()
            #else:
            
        if useLog:
            plt.yscale('log')
            plt.xscale('log')
            #accumVecHist += stdPercHist
            #sns.kdeplot(accumVec,bins=100)
            #ax.plot(binCenters,accumVecHist)
        plt.show()

In [2]:
fnMainDir = './DATA/L2/MC-AnalyticComp/Analytic/p0p30_1/'
fnMainID = 'S2003182003236'
fnAllSfx = '.L2'
fnNoiseDir = 'Perturbed/'
fnNsSfx = '_wiggledBand_'
baseLinePath = fnMainDir+fnMainID + fnAllSfx

In [ ]:

ds = nc.Dataset(baseLinePath)

In [ ]:
gv = ds.groups['geophysical_data'].variables

In [ ]:
lt=gv['Lt_412'][:]

In [ ]:
rrs=gv['Rrs_412'][:]
type(rrs)

In [3]:
swfp3=SwfNoise(baseLinePath,fnNoiseDir,multiDimVar=True)

In [ ]:
print swfp3.rrsUncDict['412']

In [4]:
swfp3.ReadFromSilent()

In [5]:
swfp3.BuildUncs('_wiggledBand_')

In [8]:
swfp3.rrsUncDict['412'].keys()

['Lt_490',
 'Lt_670',
 'Lt_443',
 'Lt_865',
 'Lt_555',
 'Lt_412',
 'Lt_765',
 'Lt_510']

In [ ]:
std443 = np.sqrt(swfp3.rrsVarDict['443'])
std490 = np.sqrt(swfp3.rrsVarDict['490'])
std510 = np.sqrt(swfp3.rrsVarDict['510'])
std555 = np.sqrt(swfp3.rrsVarDict['555'])
std670 = np.sqrt(swfp3.rrsVarDict['670'])

In [ ]:
dsMC = nc.Dataset('./DATA/L2/S2003182003236_silent_MC400.L2')
std412MC = np.array(dsMC.groups['geophysical_data'].variables['Rrs_unc_412']).flatten()

In [ ]:
badval=-32767.0
std412MC[std412MC == badval] = np.nan

In [ ]:
def JointHexPlot(detData,mcData,band,savePlot=False,showPlot=True):
    
    x_fin_idx = np.isfinite(detData)
    y_fin_idx = np.isfinite(mcData)
    fin_idx = x_fin_idx & y_fin_idx
    x_det= detData[fin_idx]
    y_mc = mcData[fin_idx]
    
    xmn,xmx = x_det.min(),x_det.max()
    ymn,ymx = y_mc.min(),y_mc.max()
    
    sns.jointplot(x_det,y_mc,xlim = (xmn,xmx),ylim=(ymn,ymx),kind='hex')
    plt.xlabel('Deterministic ($sr^{-1}$)')
    plt.ylabel('MC $sr^{-1}$')
    title = str(band)
    plt.title(title)
    if savePlot:
        savename='Comp' + title + '.pdf'
        plt.savefig(savename)
        try:
            subprocess.call(['pdfcrop', savename, savename])
        except:
            print 'This feature will only work if pdfcrop is available'
    if showPlot:
        plt.show()

In [ ]:
JointHexPlot(std412[:,-1],std412MC,'412')

In [ ]:
std443MC = np.array(dsMC.groups['geophysical_data'].variables['Rrs_unc_443']).flatten()
JointHexPlot(std443[:,-1],std443MC,'443')

In [ ]:
std490MC = np.array(dsMC.groups['geophysical_data'].variables['Rrs_unc_490']).flatten()
JointHexPlot(std490[:,-1],std490MC,'490')

In [ ]:
std510MC = np.array(dsMC.groups['geophysical_data'].variables['Rrs_unc_510']).flatten()
JointHexPlot(std510[:,-1],std510MC,'510')

In [ ]:
std555MC = np.array(dsMC.groups['geophysical_data'].variables['Rrs_unc_555']).flatten()
JointHexPlot(std555[:,-1],std555MC,'555')

In [ ]:
std670MC = np.array(dsMC.groups['geophysical_data'].variables['Rrs_unc_670']).flatten()
JointHexPlot(std670[:,-1],std670MC,'670')

In [ ]:
rrs412 = swfp3.rrsBslDict['412']
rrs443 = swfp3.rrsBslDict['443']
rrs490 = swfp3.rrsBslDict['490']
rrs510 = swfp3.rrsBslDict['510']
rrs555 = swfp3.rrsBslDict['555']
rrs670 = swfp3.rrsBslDict['670']

In [ ]:
#maxes=9*[500]
maxes=[1000,1,1,1,1,2,4,5,1000]
#maxes = 8*[10]+[500]
#maxes = 9*[10]
#maxes = [500]+4*[1]+[2,3,5,500]
rd = {i:[0,v] for i,v in zip(range(0,9),maxes)}

In [ ]:
# use jointGrid instead for greater control
sns.jointplot(std412[:,-1],std412MC,xlim=(xmn,xmx),ylim=(ymn,ymx),kind='hex');
plt.xlabel('Deterministic ($sr^{-1}$)')
plt.ylabel('MC $sr^{-1}$')
plt.savefig('Comp412.pdf')
plt.show()

In [ ]:
sns.jointplot(std412[:,-1],std412MC,xlim=(0,0.025),ylim=(0.0005,0.003),kind="hex");
plt.show()